# <mark> <b> > 1. </b> Mapeamento e analise do  Pipeline </mark>

<b>1_map_analise_pipeline_v0.ipynb </b> |  Atual notebook com as funçoes de mapeamento e analise do pipeline

### Modules & config

In [4]:
import os
import sys
import shutil
import platform
import subprocess
from io import StringIO
from pathlib import Path
from urllib import response

# from outlook_msg import Message
# import extract_msg
# import zipfile
# from pyunpack import Archive
# import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import PyPDF2

import csv
import json
#import pandas as pd

import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

#import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

#import pytesseract

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Modulos da solucao
import modules.cronometro as cron
import modules.utils as utl
import modules.trata_pdf as tpdf


In [6]:
import spacy
from spacy import displacy
import pandas as pd
from spacy.tokens import Span
from spacy.matcher import Matcher
from spacy.tokens import Token
from spacy.language import Language

nlp = spacy.load("pt_core_news_sm")

ner = nlp.remove_pipe('ner')
ruler = nlp.add_pipe("entity_ruler")

In [ ]:
# 6. XXX Nome do caminha para dict Tipo de documento
config_tipo_doc_path = "data/tipo_documentos/tipo_documento.json"

In [ ]:
tipo_documento_dict = {}

tipo_documento_dict = {
    'diario_mesquita': {
            'descricao': 'Diario oficial de Mesquita',
            'matcher_pattern_path': path_matches_pattern_json,
            'matcher_pattern_dict': matcher_pattern_dict,
            'entity_ruler_pattern_path': entityruler_file_path,
            'entity_ruler_colors': colors,
            'entity_ruler_patterns': patterns,
            'sample_content': """27/07/2023, 15:12 Nota Fiscal de Serviços Eletrônica (NFSe) https://nfe.mesquita.rj.gov.br 1/1 PREFEITURA MUNICIPAL DE MESQUITA SECRETARIA MUNICIPAL DA FAZENDA NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Número da Nota: 20234 Competência: Julho/2023 Data e Hora da Emissão: 27/07/2023 15:11:00 Código Verificação: 4FDA9FBAE PRESTADOR DE SERVIÇOS CPF/CNPJ: 50.921.369/0001-05 Inscrição Municipal: 952538 Telefone: 2297268232.. Inscrição Estadual: Nome/Razão Social: MEDSORIA CLINICA DE AVALIACAO MEDICA E PSICOLOGICA DO TRAFEGO DE MESQUITA LTDA Nome de Fantasia: Endereço: RUA PROCOPIO ,631 LOJA A ,SANTO ELIAS - Mesquita-RJ E-mail: LARA_VSORIA@HOTMAIL.COM TOMADOR DE SERVIÇOS CPF/CNPJ: 06.047.087/0033-16 | INSC:MUNICIPAL: RG: Telefone: Inscrição Estadual: Nome/Razão Social: REDE D'OR SAO LUIZ S.A. Endereço: OLINDA ELLIS N° 93 BAIRRO: CAMPO GRANDE CIDADE: RIO DE JANEIRO - RJ CEP: 23045160 E-mail: Não Informado DISCRIMINAÇÃO DOS SERVIÇOS Ref a Plantões de Janeiro, 36h no Setor de Radiologia - Médica: Lara Veiga Soria Catuladeira. VALOR TOTAL DA NOTA: R$ 4.280,16 CNAE - 8630502 - ATIVIDADE MÉDICA AMBULATORIAL COM RECURSOS PARA REALIZAÇÃO DE EXAMES COMPLEMENTARES Item da Lista de Serviços - 4.03 - HOSPITAIS, CLÍNICAS, LABORATÓRIOS, SANATÓRIOS, MANICÔMIOS, CASAS DE SAÚDE, PRONTOS-SOCORROS, AMBULATÓRIOS E CONGÊNERES. VALOR SERVIÇOS: R$ 4.280,16 VALOR DEDUÇÃO: R$ 0,00 DESC. INCOND: R$ 0,00 BASE DE CÁLCULO: R$ 4.280,16 ALÍQUOTA: 2,01% VALOR ISS: R$ 86,03 VALOR ISS RETIDO: R$ 0,00 DESC. COND: R$ 0,00 ____________________________________________________________________ VALOR PIS: R$ 0,00 VALOR COFINS: R$ 0,00 VALOR IR: R$ 0,00 VALOR INSS: R$ 0,00 VALOR CSLL: R$ 0,00 OUTRAS RETENÇÕES: R$ 0,00 VALOR LÍQUIDO: R$ 4.280,16 DADOS COMPLEMENTARES OUTRAS INFORMAÇÕES / CRITICAS EXIGIBILIDADE ISS Exigivel REGIME TRIBUTAÇÃO Sociedade Limitada SIMPLES NACIONAL Sim ( 2,01% ) ISSQN RETIDO Não LOCAL. PRESTAÇÃO SERVIÇO Mesquita - RJ LOCAL INCIDÊNCIA Mesquita - RJ Observação: LEI DA TRANSPARÊNCIA FISCAL NR. 12.741, DE 8 DE DEZEMBRO DE 2012. - PRESTADOR OPTANTE DO SIMPLES NACIONAL (ALÍQUOTA: 2,01 %) ESTA NFS-E FOI EMITIDA EM SUBSTITUIÇÃO À NFS-E 20231 Valor Aproximado dos Tributos Federais R$ 575,68 (Alíq 13,45), Tributos Estaduais R$ 0,00 (Alíq 0,00 IBPT) e Municipal de R$ 95,45 (Alíq IBPT 2,23 IBPT)""",
            'official_name': 'Official Document Name',
            'min_score_similarity_analise': 0.60,
            'min_score_similarity_documento': 0.7
    }
}  

In [19]:
# Tipo de documento em uso
tipo_doc_work = 'nfs_e'

#### Config - Mapeamento e Analise do pipeline

# 1. XXX Path para planilha de processamento de batches
conf_export_plan_path = 'processamentos/processamento_batches/df_conf_export_batch.xlsx'

# 2. XXX  Tratando nome de carga do df_processamento
map_analise_path = "processamentos/mapeamento_analise"

# 3. XXX  prefixo de nome do arquivo de exportaçao
df_root_pipe_file = "df_root_"

# 4. XXX Tipos de documentos para extracao
tipo_documento_path = "config/tipo_documentos"

# 5. XXX Path para tipo de documento patterns
tipo_documento_patterns_path = "config/tipo_documentos/patterns"


# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"


# Paths de trabalho para Raster_PDF
raster_process_pdf_path = 'processamentos/temp/pdf'
raster_process_txt_path = 'processamentos/temp/txt'





# 6. IMPORTANTE - MUDOU - Path para gestao de imagens resized
image_resized_path = "processamentos/temp/images/processadas"


#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

In [19]:
# Salva do tipo de documento em arquivo
def save_tipo_doc_to_file(dic, filename):
    with open(filename, 'w') as f:
        json.dump(dic, f) 
        
# carrega o tipo de documento do arquivo        
def load_dict_from_file(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Salva o dict doc_content em arquivo
def save_doc_content_to_file(doc_content):
    file_doc_content_path = os.path.join(map_analise_path, 'doc_content_' + batch_name + ".json")
    with open(file_doc_content_path, 'w') as f:
        json.dump(doc_content, f)
        
# Carrega o dict doc_content em arquivo
def load_doc_content_from_file():
    file_doc_content_path = os.path.join(map_analise_path, 'doc_content_' + batch_name + ".json")
    with open(file_doc_content_path, 'r') as f:
        return json.load(f)              

### Funcoes para Analise do pipeline

In [ ]:
novo_tipo_documento = {
    'lista_nota': {
            'descricao': 'Listagem de Notas Fiscais',
            'matcher_pattern_path': path_matches_pattern_json,
            'matcher_pattern_dict': matcher_pattern_dict,
            'entity_ruler_pattern_path': entityruler_file_path,
            'entity_ruler_colors': colors,
            'entity_ruler_patterns': patterns,
            'sample_content': """02/08/23, 16:33 Nota Fiscal de Serviços Eletrônica (NFSe) Número da Nota: PREFEITURA MUNICIPAL DE MAGE ia Competência: SECRETARIA MUNICIPAL DA FAZENDA Agosto/2023 NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Data e Hora da Emissão: 02/08/2023 16:29:00 Código Verificação: OCDD6FB51 PRESTADOR DE SERVIÇOS CPF/CNPJ: Inscrição Municipal: Telefone: Inscrição Estadual: 33.462.862/0001-95 li: 4337 al Nome/Razão Social: SÃO MARCOS TERRAPLENAGEM E CONSTRUÇÃO LTDA. Nome de Fantasia: Endereço: RUA DAS MARGARIDAS ,578 ,SANTA DALILA - Magé-RJ E-mail: TOMADOR DE SERVIÇOS CPF/CNPJ: 29.115.474/0001-60 HINSC:MUNICIPAL: RG: | Telefone: Inscrição Estadual: Nome/Razão Social: MUNICÍPIO DE MACAE Endereço: ; PRESIDENTE FELICIANO SODRE Nº 534 PAÇO E-mail: NÃO INFORMADO MUNICIPAL BAIRRO: CENTRO CIDADE: MACAE ; - RJ CEP: 27913080 DISCRIMINAÇÃO DOS SERVIÇOS Serviço de Recomposição do Pavimento Asfáltico (Tapa Buraco), na cidade de Macaé/RJ, abrangendo Perímetro Urbano, Região Serrana, conf Boletim de Medição 08 — CO 022/2022 — Contrato 045/2022 - . Empenhos 501/2023 e 502/2023. M. Obra (10%)= R$ 104.468,45 Retenção p/ Prev. Social=R$ 11.491,53 Banco: ITAU Agência: 0726 C/Corrente: 35410-3 VALOR TOTAL DA NOTA: R$ 1.044.684,55 CNAE - 4313400 - OBRAS DE TERRAPLENAGEM . é o Item da Lista de Serviços - 7.02 - EXECUÇÃO, POR ADMINISTRAÇÃO, EMPREITADA OU SUBEMPREITADA, DE OBRAS DE CONSTRUÇÃO CIVIL, HIDRAULICA OU ELETRICA E DE OUTRAS OBRAS SEMELHANTES, INCLUSIVE SONDAGEM, PERFURAÇÃO DE POÇOS, ESCAVAÇÃO, DRENAG = [m]\' vm La [=] VALOR SERVIÇOS: R$ 1.044.684,55 VALOR | DEDUÇÃO: R$ 0,00 DESC. INCOND: BASE DE R$ 0,00 CALCULO: R$ 1.044.684,55 ALÍQUOTA: 2% VALOR ISS: R$ 0,00 VALOR PIS: R$ 0,00 VALOR COFINS: R$ 0,00 VALOR IR: R$ 0,00 VALOR CSLL: R$ 0,00 OUTRAS R$ 0,00 eEd Era a e [=] Lt A =" DADOS VALOR INSS: R$ 11.491,53 VALOR ISS RETIDO: R$ 20.893,69 RETENÇÕES: DESC. COND: R$ 0,00 VALOR LÍQUIDO: R$ 1.012.299,33 COMPLEMENTARES OUTRAS INFORMAÇÕES / CRITICAS EXIGIBILIDADE ISS Exigivel REGIME TRIBUTAÇÃO Sociedade Limitada SIMPLES NACIONAL Não ISSQN RETIDO Sim LOCAL. PRESTAÇÃO SERVIÇO Macaé - RJ LOCAL INCIDÊNCIA Macaé - RJ Observação: - ISS RETIDO PELO TOMADOR DE SERVIÇOS CNPJ: 29.115.474/0001-60 Valor Aproximado dos Tributos Federais R$ 136154,26 (Alíq 13,45), Tributos Estaduais R$ 0,00 (Alíq 0,00 IBPT) e Municipal de R$ 49906,36 (Alíq IBPT 4,93 IBPT) https://nfs-e.mage.rj.gov.br""",
            'official_name': 'Official Document Name',
            'min_score_similarity_analise': 0.45,
            'min_score_similarity_documento': 0.6
    }
}

tipo_documento_dict.update(novo_tipo_documento)

In [ ]:
# XXX Salvando o dict do tipo de documento para arquivo json
save_tipo_doc_to_file(tipo_documento_dict, config_tipo_doc_path)

In [22]:
# 0. XXX IMPORTANTE, carregar o dict de tipo de documento 
tipo_documento_dict = load_dict_from_file(config_tipo_doc_path)

In [8]:
# Unica funcao que devera ser movida para junto das demais funcoes de dicionario
def define_rotulo_acao(nome_arquivo):
    
    for palavra_chave, rotulo in mapeamento_palavras_chave.items():
        if palavra_chave.lower() in nome_arquivo.lower():
            break
    else:
        rotulo = 'prov_nota_fiscal' #"sem_rotulo"
        palavra_chave = 'default'
        acao_sugerida = sugestoes_acao.get(rotulo, 'None')
        return palavra_chave, rotulo, acao_sugerida
        # palavra_chave = 'None' #"sem_palavra_chave"
        # acao_sugerida = 'None' #"sem_acao_sugerida"
        
        return palavra_chave, rotulo, acao_sugerida
        #print(f'nome_arquivo: {nome_arquivo} | rotulo: {rotulo}')
    if rotulo != 'None': #"sem_rotulo"
        acao_sugerida = sugestoes_acao.get(rotulo, 'None') # "Ação não definida"
        return palavra_chave, rotulo, acao_sugerida
    
    
    
    # Dicionário para mapear palavras-chave a rótulos
mapeamento_palavras_chave = {
    "relatorio": "prov_relatorio",
    "listagem": "prov_listagem",
    "NF": "prov_nota_fiscal",
    "nf": "prov_nota_fiscal",
    "relatorio": "prov_listagem",
    "sintetico": "prov_listagem",
    "livro": "prov_livro_registro",
    "sintético": "prov_listagem",
    "nota": "prov_nota_fiscal",
    "zip": "doc_zip",
    "rar": "doc_rar",
    "valores": "prov_dinheiro",
}

# Dicionário mapeando rótulos a ações sugeridas
sugestoes_acao = {
    "prov_relatorio": "NO_PROCESS",
    "prov_listagem": "NO_PROCESS",
    "prov_nota_fiscal": "PROCESS",
    "sem_rotulo": "MANUAL_REV",
    "prov_livro_registro": "NO_PROCESS",
    "doc_nao_pdf": "verificar",
    "nao_pdf": "NO_PROCESS",
    "doc_zip": "NO_PROCESS",
    "pdf_mul_paginas": "SPLIT",
}



# 2.Testando
nome_arquivo = 'batatinha_quando_nasce.pdf' # 'pre-processamento'
#palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo, debug)
#print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    

## <mark> <b> 3.0</b>  Mapeamento e Analise do pipeline </mark>

In [ ]:
# 1. XXX apagar arquivos zone
utl.apagar_zone(documentos_extracao_path)

In [9]:
# SOMENTE PARA TESTE DE FUNCAO
#fake_parent_document_unique_id = 'f976c128-1f41-4551-bffd-fac687c1c8b2'

# Busca proximo Batch caso nao esteja rodando email
batch_name = utl.busca_proximo_batch(conf_export_plan_path)
batch_name

'Batch_23'

In [2]:
# 0.B XXX Funçao de criacao do PDF Pesquisavel
def run_ocrmypdf(input_file, output_file):
    command = [
        'ocrmypdf',
        '--language', 'por',
        '--deskew',
        '--force-ocr',
        input_file,
        output_file
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if result.returncode == 0:
        print(f"OCRmyPDF completed successfully. Output saved to {output_file}.")
    else:
        print(f"OCRmyPDF failed with error: {result.stderr.decode('utf-8')}")
        
        
# 1.A XXX Extracao de texto de todo o documento - PDF PESQUISAVEL	
def extracao_texto_PDF_P(i, document_unique_id, nome_arquivo, file_path, debug):    
    
   # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Extrair texto dentro do retângulo
    text_P = page.get_text("text")
    
    pdf_document.close()
    
    texto_PDF_P = re.sub('\s+', ' ', text_P).strip()  
    if debug:
        print(f'\nFUNC analise_texto_PDF: doc.:{nome_arquivo}   texto_PDF_P: \n\n{texto_PDF}\n\n')

    return texto_PDF_P  


# 1.B XXX Extracao de texto de todo o documento - RASTER PDF
def extracao_texto_Raster_P(i, document_unique_id, nome_arquivo, file_path, debug):    
    
    output_file = None
    txt_document_file = None
    
    input_file = file_path
    
    #output_file = "/home/dani-boy/extractNF/processamentos/temp/documento.pdf"
    output_file = os.path.join(raster_process_pdf_path, document_unique_id + '.pdf')

    # 1. XXX Executar o comando OCRmyPDF
    run_ocrmypdf(input_file, output_file)

    txt_document_file = os.path.join(raster_process_txt_path, document_unique_id + '.txt')

    txt_dir = os.path.dirname(txt_document_file)
    if not os.path.exists(txt_dir):
        os.makedirs(txt_dir)

    # Execute o comando
    subprocess.run(["pdftotext", output_file, txt_document_file])

    # 3. XXX Ler o arquivo TXT
    with open(txt_document_file, 'r', encoding='utf-8') as arquivo:
        texto_OCR_R = arquivo.read()
    texto_Raster_P = re.sub('\s+', ' ', texto_OCR_R).strip()
    
    os.remove(output_file)
    os.remove(txt_document_file)
    
    return texto_Raster_P  


# 2. XXX Analise de silimaridade doc1 x doc2
def analisa_similiaridade_doc(doc1, doc2):
    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([doc1, doc2])

    # Calculando Similaridade de Cosseno
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])

    print(f"Score de Similaridade: {similarity_scores[0][0]}")
    
    return similarity_scores[0][0]    

In [1]:
# 1.XXX  Acao 1 - Ler todo o pipeline de documentos recebidos - ESSA E A UNICA FUNCAO QUE ITERA NO DIRETORIO
def scan_pipeline_documentos(tipo_documento_dict, tipo_doc_work, documentos_extracao_path, batch_name, fase, atividade, status, debug):
    
    doc_info = {}
    resumo = {}
    raw_document = []
    doc_content = {}
    
    doc1 = tipo_documento_dict.get(tipo_doc_work, {}).get('sample_content', 'valor_padrao')
    score_min_analise = tipo_documento_dict.get(tipo_doc_work, {}).get('min_score_similarity_analise', 'valor_padrao')
    score_min_documento = tipo_documento_dict.get(tipo_doc_work, {}).get('min_score_similarity_documento', 'valor_padrao')
    
    output_dir = os.path.join(documentos_extracao_path, batch_name)
    i = 1
    for root, dirs, files in os.walk(output_dir):
        folder_name = os.path.basename(root)
        #print(folder_name)
        for file in files:
            pdf_pesquisavel_map = None
            document_unique_id = utl.generate_unique_id()
            date_time = cron.timenow_pt_BR(),
            nome_arquivo = file
            palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo)
            acao_executada = "Analise"
            informations = ' '    
            file_name = file.lower()    
            file_path = os.path.join(root, file)
            file_hash = utl.generate_file_hash(file_path)
            if file.lower().endswith('.zip') or file.lower().endswith('.rar') or file.lower().endswith('.7z'):
                compressed_file_path = file_path
                compressed_file_name = os.path.basename(compressed_file_path)
                compressed_file_hash = utl.generate_file_hash(compressed_file_path)
                compressed_file_unique_id = utl.generate_unique_id()
                parent_document_unique_id = compressed_file_unique_id
                level = 2
            new_path_name = os.path.join(output_dir, file)
            acao_sugerida = 'ARCHIEVE_EXTRACTION'
            nro_pages = None
            one_page = False
            doc2 = None
            score = None
            
            if file.lower().endswith('.pdf'):
                doc_one_page, nro_pages = tpdf.analisa_nro_pages(file_path)
                if nro_pages:
                        
                    one_page = doc_one_page
                    
                    if nro_pages > 1:
                        acao_sugerida = 'SPLIT_PAGES'
                    else:
                        texto_PDF_P = extracao_texto_PDF_P(i, document_unique_id, nome_arquivo, file_path, debug)  
                        level = 3
                        if texto_PDF_P:
                            doc2 = texto_PDF_P
                            pdf_pesquisavel_map = True
                        else:
                            pdf_pesquisavel_map = False  
                    
                        if pdf_pesquisavel_map == False:
                            texto_Raster_P = extracao_texto_Raster_P(i, document_unique_id, nome_arquivo, file_path, debug)
                            if texto_Raster_P:
                                doc2 = texto_Raster_P
                        
                        score = analisa_similiaridade_doc(doc1, doc2)
                        
                        if score > score_min_documento:
                            acao_sugerida = 'PREPROCESS_EXTRACT'
                            if debug:
                                print(f'\nQuase certeza que e NFS-e pois seu score e: {score} e o score minimo documento e de: {score_min_documento} - acao_sugerida: {acao_sugerida}\n')
                        elif score > score_min_analise: 
                            acao_sugerida = 'PREPROCESS_ANALISE'
                            if debug:
                                print(f'\nPode ser que se trate de um documento NFS-e pois seu score e: {score} e o score minimo e de analise e: {score_min_analise} - acao_sugerida: {acao_sugerida}\n')   
                        elif score < score_min_analise:
                            acao_sugerida = 'NO_PROCESS'
                            if debug:
                                print(f'\nE pouco provavel que seja NFSe pois seu score e de: {score} para um score minimo de: {score_min_analise} - acao_sugerida: {acao_sugerida}\n')
      
                else:
                    one_page = False
                    doc2 = None
                    score = None

            diretorio = os.path.basename(file_path)
            if folder_name == batch_name:
                folder_name = "root_dir"
            
            if debug:    
                print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    
            
            documento_info = {
                "seq": i,
                "date_time": cron.timenow_pt_BR(),
                "sigla_tipo": tipo_doc_work,
                "batch": batch_name,
                "original_file_name": file,
                "directory": folder_name,
                "file_path": file_path,
                "pdf_pesquisavel": pdf_pesquisavel_map,
                "level": level,
                "score": score,
                "action_item": acao_sugerida,
                "file_hash": file_hash,
                "content": doc2
            }
            doc_content[document_unique_id] = documento_info
            
            new_row = {
                "seq": i,
                "date_time": cron.timenow_pt_BR(),
                "sigla_tipo": tipo_doc_work,
                "batch": batch_name,
                "fase_processo": fase,
                "nome_atividade": atividade,
                "status_documento": status,
                "acao_executada": acao_executada,
                "original_file_name": file,
                "directory": folder_name,
                "one_page": one_page,
                "pages": nro_pages,
                "pdf_pesquisavel": pdf_pesquisavel_map,
                "score": score,
                "palavra_chave": palavra_chave,
                "document_tag": rotulo,
                "action_item": acao_sugerida,
                "level": level,
                "pdf_pesquisavel": pdf_pesquisavel_map,
                "document_unique_id": document_unique_id,
                "parent_document_unique_id": parent_document_unique_id,
                "file_hash": file_hash,
                "file_path": file_path,
                "informations": informations,
            }
            raw_document.append(new_row)
            
            if debug:
                print(f'seq: {i} | file: {file}')
            i += 1
            
    df_trans_pipe = pd.DataFrame(raw_document)
    
    return df_trans_pipe, doc_content

In [12]:
# 1. XXX Crio o DF df_scan_pipe
fase = 'analise' # 'pre-processamento'
atividade = 'scan_analise'
status = 'root_analise'
doc_content = {}
documentos = []
debug = False

df_root_pipe, doc_content = scan_pipeline_documentos(tipo_documento_dict, tipo_doc_work, documentos_extracao_path, batch_name, fase, atividade, status, debug)

OCRmyPDF completed successfully. Output saved to processamentos/temp/pdf/27df9e70-b5fb-45b7-8f59-0c04ed9728e2.pdf.
Score de Similaridade: 0.7164629052589963
Score de Similaridade: 0.9328603069007027
Score de Similaridade: 0.9325618827215076
Score de Similaridade: 1.0
Score de Similaridade: 0.9361023204633862
Score de Similaridade: 0.9349205511347822
Score de Similaridade: 0.9325618827215076
Score de Similaridade: 0.8179155457544696
Score de Similaridade: 0.8098129472961368
Score de Similaridade: 0.4737137444296768
Score de Similaridade: 0.42334416430218325


In [13]:
# 0. Acertando o Index
df_root_pipe.set_index('document_unique_id', inplace=True)
df_root_pipe

,seq,date_time,batch,sigla_tipo,fase_processo,nome_atividade,status_documento,acao_executada,original_file_name,directory,...,pdf_pesquisavel,score,palavra_chave,document_tag,action_item,level,parent_document_unique_id,file_hash,file_path,informations
document_unique_id,,,,,,,,,,,,,,,,,,,,,
4433a03b-d30a-4c92-80fc-015912e1f357,1,"(26/09/2023 16:16:39,)",Batch_23,nfs_e,analise,scan_analise,root_analise,Analise,MESQUITA_PDF_31282023_2258.zip,root_dir,...,None,NaN,zip,doc_zip,ARCHIEVE_EXTRACTION,2,511c5820-d1d1-4c39-bfd6-6f6df5975fd4,8d7038d712373364fa4c7680a887a0ceed01c8692d6958...,pipeline_extracao_documentos/2_documentos_para...,
27df9e70-b5fb-45b7-8f59-0c04ed9728e2,2,"(26/09/2023 16:16:39,)",Batch_23,nfs_e,analise,scan_analise,root_analise,Analise,1.pdf,teste,...,False,0.716463,default,prov_nota_fiscal,PREPROCESS_EXTRACT,3,511c5820-d1d1-4c39-bfd6-6f6df5975fd4,66a7db9ee1500d5f9fa5da26563cfd7b68f1f5ba3daba2...,pipeline_extracao_documentos/2_documentos_para...,
393a4eab-4b10-48e4-8d06-2fecadfa3b48,3,"(26/09/2023 16:16:50,)",Batch_23,nfs_e,analise,scan_analise,root_analise,Analise,Livro de Registro do ISSQN.pdf,115964,...,None,NaN,livro,prov_livro_registro,SPLIT_PAGES,3,511c5820-d1d1-4c39-bfd6-6f6df5975fd4,b960962503987f6e05f5646d71a789facfe4e80ccb8890...,pipeline_extracao_documentos/2_documentos_para...,
ab2457b7-ea5c-4191-acf0-bc8edc04879e,4,"(26/09/2023 16:16:50,)",Batch_23,nfs_e,analise,scan_analise,root_analise,Analise,2023 -5.pdf,159871,...,True,0.932860,default,prov_nota_fiscal,PREPROCESS_EXTRACT,3,511c5820-d1d1-4c39-bfd6-6f6df5975fd4,23a28a363c2d2c8b700ac4775164f7c0f0e2d6cef6166d...,pipeline_extracao_documentos/2_documentos_para...,
b6b5af8f-78dc-4627-9322-9c3b70f46a48,5,"(26/09/2023 16:16:50,)",Batch_23,nfs_e,analise,scan_analise,root_analise,Analise,2023 -7.pdf,159871,...,True,0.932562,default,prov_nota_fiscal,PREPROCESS_EXTRACT,3,511c5820-d1d1-4c39-bfd6-6f6df5975fd4,54045f4c09341d9f8d69438e7afe71eff46bb4e731392b...,pipeline_extracao_documentos/2_documentos_para...,
d6b2bfd0-f5ba-4678-a8d7-9d3a16cb7c6c,6,"(26/09/2023 16:16:50,)",Batch_23,nfs_e,analise,scan_analise,root_analise,Analise,2023 -4.pdf,159871,...,True,1.000000,default,prov_nota_fiscal,PREPROCESS_EXTRACT,3,511c5820-d1d1-4c39-bfd6-6f6df5975fd4,ddd09bb806dc79e98a74c5cf1adc6a5bd23ea1b4f1bfa7...,pipeline_extracao_documentos/2_documentos_para...,
ef2477eb-67ca-43bf-bab4-826d872d47e7,7,"(26/09/2023 16:16:50,)",Batch_23,nfs_e,analise,scan_analise,root_analise,Analise,2023 -6.pdf,159871,...,True,0.936102,default,prov_nota_fiscal,PREPROCESS_EXTRACT,3,511c5820-d1d1-4c39-bfd6-6f6df5975fd4,8910a092d3aa53b6a1eb805c783245493760a1c46dadfa...,pipeline_extracao_documentos/2_documentos_para...,
fd74ace6-6582-4e71-ada9-f21760846ade,8,"(26/09/2023 16:16:50,)",Batch_23,nfs_e,analise,scan_analise,root_analise,Analise,2023 -3.pdf,159871,...,True,0.934921,default,prov_nota_fiscal,PREPROCESS_EXTRACT,3,511c5820-d1d1-4c39-bfd6-6f6df5975fd4,25387d066a46925acba7ddfe3cd97e2e70f92838ef4312...,pipeline_extracao_documentos/2_documentos_para...,
c8a7cd74-931c-4b38-814f-779874d69417,9,"(26/09/2023 16:16:50,)",Batch_23,nfs_e,analise,scan_analise,root_analise,Analise,2023 -8.pdf,159871,...,True,0.932562,default,prov_nota_fiscal,PREPROCESS_EXTRACT,3,511c5820-d1d1-4c39-bfd6-6f6df5975fd4,8be881c377bf3064ef61f014fd14602cc736a73b69de8c...,pipeline_extracao_documentos/2_documentos_para...,


### exportando o map_analise_path

In [14]:
# 1. XXX Definiçao do path para salvar o arquivo
file_path_root_pipe = os.path.join(map_analise_path, df_root_pipe_file + batch_name + ".xlsx")

# 2. XXX Salvando o arquivo de df: df_root_pipe
df_root_pipe.to_excel(file_path_root_pipe, index=True)

In [15]:
# 3. XXX Processo de salva o dict para arquivo
save_doc_content_to_file(doc_content)

## Pesquisas e outros

In [ ]:
# Modelo a ser utilizado para busca de dados do dict doc_content
texto = doc_content.get(document_unique_id, {}).get('content', 'valor_padrao')

In [ ]:
# Modelo a ser utilizado para carregar o dict doc_content
doc_content = {}
doc_content = load_doc_content_from_file()

### Atualizando o valor da coluna status_documento

In [ ]:
def update_status(row):
    if row['seq'] in sequencia:
        row['status_documento'] = 'PREPROCESS_EXTRACT'
    return row

In [ ]:
sequencia = [2, 8, 6]

In [ ]:
df_root_pipe = df_root_pipe.apply(update_status, axis=1)
df_root_pipe

In [ ]:
# Podemos chegar de forma mais rapida
subset_df.loc[subset_df['status_documento'] == "Template_encontrado", 'pdf_pesquisavel'] = False

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq == 59')

In [ ]:
# XXX Salvando o DF (IMPORTANTE)
df_analise_pipe.to_excel("df_mapeamento_e_analise2.xlsx", index=True)

In [ ]:
# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')

In [ ]:
# 7. XXX Retiro o indice do DF - Resetando o índice e mantendo o índice original como uma nova coluna
df_analise_pipe.reset_index(inplace=True)


# XXX Criando uma nova coluna no DF
df_conferencia.insert(loc=50, column='original_file_name', value=df_conferencia['file_path'].apply(lambda x: os.path.basename(x)))

df['coluna_cnae'] = df['coluna_cnae'].apply(lambda x: x.strip() if isinstance(x, str) else x)


# 4. XXX Usando loc para Filtrar Baseado em one_page == False
df_pages_2_split = df_root_pipe[df_root_pipe['one_page'] == False]



# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')


# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)


# 4. Filtrando Linhas Baseadas em Valores em uma Lista

valores = [11, 16, 30, 41]
subset_df = df_scan_pipe[df_scan_pipe['seq'].isin(valores)]

# 2. Usando iloc para Filtrar um Número Específico de Linhas
subset_df = df_scan_pipe.iloc[:10]




registro_específico = df_scan_pipe.loc['e1f4b1af-30f3-45d2-85a7-1bb895bd5325']



df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: row['coluna1'] * 2 if row['coluna2'] > 0 else row['coluna1'], axis=1)


# Atualizei o DF baseado em condiçao de outra coluna
df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: False if row['status_documento'] == "Template_encontrado" else row['pdf_pesquisavel'], axis=1)


# Podemos chegar de forma mais rapida
df_analise_pipe.loc[df_analise_pipe['status_documento'] == "Template_encontrado", 'pdf_pesquisavel'] = False


# 8. XXX Concatenando os DataFrames
df_analise_pipe = pd.concat([df_analise_pipe, df_docs_splitados], ignore_index=True)

# 9. XXX Volto novamente o indice do DF
df_analise_pipe.set_index('document_unique_id', inplace=True)